In [1]:
%matplotlib inline

import itertools
from tqdm import tqdm
from tqdm import trange
from functools import partial
from multiprocessing import Pool, cpu_count
import numpy as np
# np.seterr(all='ignore')
import scipy.special as special
import scipy.integrate as integrate
import scipy.optimize as optimize
import scipy.interpolate as interpolate
from scipy.stats import rv_continuous, poisson, uniform, norm, chi2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import wf_func as wff

Ncpu = 100
window = 1029

In [4]:
def glow(n, tau):
    return np.random.exponential(tau, size=n)

def transit(n, sigma):
    return np.random.normal(0, sigma, size=n)

def time(n, tau, sigma):
    if tau == 0:
        return np.sort(transit(n, sigma))
    elif sigma == 0:
        return np.sort(glow(n, tau))
    else:
        return np.sort(glow(n, tau) + transit(n, sigma))

def convolve_exp_norm(x, tau, sigma):
    if sigma == 0.:
        y = np.where(x >= 0., 1/tau * np.exp(-x/tau), 0.)
    elif tau == 0.:
        y = norm.pdf(x, loc=0, scale=sigma)
    else:
        alpha = 1/tau
        co = alpha/2. * np.exp(alpha*alpha*sigma*sigma/2.)
        x_erf = (alpha*sigma*sigma - x)/(np.sqrt(2.)*sigma)
        y = co * np.exp(-alpha*x) * (1. - special.erf(x_erf))
    return y

def charge(n):
    gmu = 160.
    gsigma = 40.
    return np.random.normal(gmu, gsigma, size=n)

def start_time(a0, a1, mu, tau, sigma):
    np.random.seed(a0)
    stime = np.zeros(a1 - a0)
    npe = poisson.ppf(1 - uniform.rvs(scale=1-poisson.cdf(0, mu), size=a1 - a0), mu).astype(np.int)
    samples = [np.vstack((time(n, tau, sigma), charge(n))).T for n in npe]
    waves = np.empty([a1 - a0, window])
    pan = np.arange(window)
    for i in range(a1 - a0):
        waves[i] = np.sum([np.where(pan > samples[i][j, 0], convolve_exp_norm(pan - samples[i][j, 0], tau=6, sigma=2), 0) * samples[i][j, 1]] for j in range(window))
    for i in range(a1 - a0):
        t = np.around(samples[i][:, 0])
        uni, ind, cou = np.unique(t, return_index=True, return_counts=True)
        if max(cou) == 1:
            samples[i][:, 0] = t
            continue
        ind = np.append(ind, len(t))
        c = np.array([np.sum(samples[i][:, 1][ind[n]:ind[n+1]]) for n in range(len(ind)-1)])
        samples[i] = np.vstack((uni, c)).T
    for i in range(a1 - a0):
        logL = lambda t0 : -1 * np.sum(np.log(np.clip(convolve_exp_norm(samples[i][:, 0] - t0, tau, sigma), 
                                                      np.finfo(np.float).tiny, np.inf)))
        if sigma == 0.:
            stime[i] = optimize.minimize(logL, x0=np.min(samples[i][:, 0]), method='SLSQP')['x']
        else:
            stime[i] = optimize.minimize(logL, x0=np.min(samples[i][:, 0])-1, method='L-BFGS-B', bounds=[[-np.inf, samples[i][0, 0]]])['x']
    return stime, samples

def deltatime(mu, tau, sigma, N):
    chunk = N // Ncpu + 1
    slices = np.vstack((np.arange(0, N, chunk), np.append(np.arange(chunk, N, chunk), N))).T.astype(np.int).tolist()
    with Pool(min(Ncpu, cpu_count())) as pool:
        result = pool.starmap(partial(start_time, mu=mu, tau=tau, sigma=sigma), slices)

    stime = np.concatenate([result[i][0] for i in range(len(slices))])
    samples = list(itertools.chain.from_iterable([result[i][1] for i in range(len(slices))]))
        
    deltat = stime
    deltat0 = np.array([samples[i][0, 0] for i in range(N)])
    std = np.std(deltat, ddof=-1)
    std0 = np.std(deltat0, ddof=-1)
    return deltat, deltat0, std, std0

In [5]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
tau = 6
sigma = 2
t = np.arange(-20, 100, 0.1)
ax.plot(t+6, convolve_exp_norm(t, tau, sigma), color='b', label='Single PE response')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [ ]:
para = {'ideal' : {'tau' : 20., 'sigma' : 0.}, 
        'JNE' : {'tau' : 20., 'sigma' : 1.5}, 
        'JUNO' : {'tau' : 20., 'sigma' : 6.}, 
        'SK' : {'tau' : 0., 'sigma' : 3.}, 
        'slow' : {'tau' : 40., 'sigma' : 2.}, 
        'fast' : {'tau' : 10, 'sigma' : 2.}}

N = int(1e5)
Mu = np.linspace(0.5, 20, num=40)
result = {}

for key in para.keys():
    tau = para[key]['tau']
    sigma = para[key]['sigma']
    Deltaall = np.zeros((len(Mu), N))
    Delta1st = np.zeros((len(Mu), N))
    Stdall = np.zeros(len(Mu))
    Std1st = np.zeros(len(Mu))
    for m in trange(len(Mu), desc=key.ljust(6, ' ')):
        mu = Mu[m]
        Deltaall[m], Delta1st[m], Stdall[m], Std1st[m] = deltatime(mu, tau, sigma, N)
    result.update({key : {'Deltaall' : Deltaall, 'Delta1st' : Delta1st, 'Stdall' : Stdall, 'Std1st' : Std1st}})

ideal :   0%|          | 0/40 [00:00<?, ?it/s]<ipython-input-4-349730fff8e6>:40: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  waves[i] = np.sum([np.where(pan > samples[i][j, 0], convolve_exp_norm(pan - samples[i][j, 0], tau=6, sigma=2), 0) * samples[i][j, 1]] for j in range(window))
<ipython-input-4-349730fff8e6>:40: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  waves[i] = np.sum([np.where(pan > samples[i][j, 0], convolve_exp_norm(pan - samples[i][j, 0], tau=6, sigma=2), 0) * samples[i][j, 1]] for j in range(window))
<ipython-input-4-349730fff8e6>:40: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin i

In [33]:
fig = plt.figure()
fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.85, bottom=0.1, wspace=0.18, hspace=0.35)

alpha = 0.95

keys = list(para.keys())
ax = fig.add_subplot(gs[0, 0])
for k in range(len(keys)):
    key = keys[k]
    yerr1st = np.vstack([result[key]['Std1st']-np.sqrt(np.power(result[key]['Std1st'],2)*N/chi2.ppf(1-alpha/2, N)), np.sqrt(np.power(result[key]['Std1st'],2)*N/chi2.ppf(alpha/2, N))-result[key]['Std1st']])
    yerrall = np.vstack([result[key]['Stdall']-np.sqrt(np.power(result[key]['Stdall'],2)*N/chi2.ppf(1-alpha/2, N)), np.sqrt(np.power(result[key]['Stdall'],2)*N/chi2.ppf(alpha/2, N))-result[key]['Stdall']])
    ax.errorbar(Mu, result[key]['Std1st'], yerr=yerr1st, label='1st PE', marker='^')
    ax.errorbar(Mu, result[key]['Stdall'], yerr=yerrall, label='all PE', marker='^')
    ax.set_xlabel(r'$\mu$')
    ax.set_ylabel(r'$\delta/\mathrm{ns}$')
    ax.set_title(r'$\tau=${:.01f}'.format(para[key]['tau']) + r'$\mathrm{ns}\ $' + 
                 r'$\sigma=${:.01f}'.format(para[key]['sigma']) + r'$\mathrm{ns}\ $' + 
                 r'$\mathrm{N}=$' + '{0:.1E}\n'.format(N))
    ax.grid()
    if k == 0:
        ax.legend(loc='upper right')
fig.savefig('Note/figures/vs-delta.pgf')
fig.savefig('Note/figures/vs-delta.pdf')
plt.close()
    
fig = plt.figure()
fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.85, bottom=0.1, wspace=0.18, hspace=0.35)

keys = list(para.keys())
ax = fig.add_subplot(gs[0, 0])
for k in range(len(keys)):
    key = keys[k]
    yerr1st = np.vstack([result[key]['Std1st']-np.sqrt(np.power(result[key]['Std1st'],2)*N/chi2.ppf(1-alpha/2, N)), np.sqrt(np.power(result[key]['Std1st'],2)*N/chi2.ppf(alpha/2, N))-result[key]['Std1st']])
    yerrall = np.vstack([result[key]['Stdall']-np.sqrt(np.power(result[key]['Stdall'],2)*N/chi2.ppf(1-alpha/2, N)), np.sqrt(np.power(result[key]['Stdall'],2)*N/chi2.ppf(alpha/2, N))-result[key]['Stdall']])
    yerr = np.vstack([result[key]['Stdall'] / result[key]['Std1st'] - (result[key]['Stdall'] - yerrall[0]) / (result[key]['Std1st'] + yerr1st[1]), 
                      (result[key]['Stdall'] + yerrall[1]) / (result[key]['Std1st'] - yerr1st[0]) - result[key]['Stdall'] / result[key]['Std1st']])
    ax.errorbar(Mu, result[key]['Stdall'] / result[key]['Std1st'], yerr=yerr, label=r'$\frac{\delta_{1st}}{\delta_{all}}$', marker='^')
    ax.set_ylim(0.9, 1.01)
    ax.hlines(1, xmin=Mu[0], xmax=Mu[-1], color='k')
    ax.set_xlabel(r'$\mu$')
    ax.set_ylabel(r'$\delta_{all}-\delta_{1st}/\mathrm{ns}$')
    ax.set_title(r'$\tau=${:.01f}'.format(para[key]['tau']) + r'$\mathrm{ns}\ $' + 
                 r'$\sigma=${:.01f}'.format(para[key]['sigma']) + r'$\mathrm{ns}\ $' + 
                 r'$\mathrm{N}=$' + '{0:.1E}\n'.format(N))
    ax.grid()
    if k == 0:
        ax.legend(loc='lower right')
fig.savefig('Note/figures/vs-deltasub.pgf')
fig.savefig('Note/figures/vs-deltasub.pdf')
plt.close()

In [6]:
np.random.seed(3)
fig = plt.figure(figsize=(12, 8))
fig.tight_layout()
ax = fig.add_subplot()
tau = 20
sigma = 5
t = np.arange(-20, 100, 0.1)
ax.plot(t, convolve_exp_norm(t, tau, sigma), color='b', label='Time profile')
smaples = time(15, tau, sigma)
ax.vlines(smaples, ymin=0, ymax=0.01, color='C0', label='all PE')
ax.vlines(smaples.min(), ymin=0, ymax=0.02, color='C1', label='1st PE')
ax.vlines(0, ymin=0, ymax=0.05, color='r', label='truth t0')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$P(t)$')
ax.legend()
fig.savefig('Note/figures/vs-demo.pgf')
plt.close()